In [7]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch
import torch.optim as optim
from torch.autograd import Variable
import csv
from drive.minisom.minisom import MiniSom
from sklearn.metrics import accuracy_score, normalized_mutual_info_score
import drive.DEC.metrics as met
import numpy as np
import pandas as pd
from time import time
from sklearn.cluster import KMeans

Using TensorFlow backend.


In [8]:
(train_x, train_y), (val_x, val_y) = mnist.load_data()
 
train_x = train_x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.
train_x = train_x.reshape((len(train_x), np.prod(train_x.shape[1:])))
val_x = val_x.reshape((len(val_x), np.prod(val_x.shape[1:])))
print(train_x.shape)
print(val_x.shape)

11493376/11490434 [==============================] - 1s 0us/step
(60000, 784)
(10000, 784)


In [9]:
train_x = np.concatenate((train_x, val_x),0)
train_y = np.concatenate((train_y, val_y),0)

In [ ]:
train_x.shape


(70000, 784)

In [10]:
class SelfOrganizingMaps():   
    def __init__(self,x,y,input_len,sigma,lr,num_iteration,random_seed):
        self.model = MiniSom(x = x, y = y, input_len = input_len, sigma = sigma, learning_rate = lr, random_seed=random_seed)
        self.num_iteration = num_iteration
        
    def train(self, data):    
        self.model.train_random(data = data, num_iteration = self.num_iteration)
        return self.model
    
    def predict(self, valx, trainx, trainy, classes):
        mappings = self.model.labels_map(trainx, trainy)
        dataXpred  = []
        for i in range(len(valx)):
            wt = self.model.winner(valx[i])
            cluster = mappings[wt]            
            clssCount = []                      
            for cls in classes: 
              clssCount.append(cluster.count(cls))  
            Predictlabel = np.argmax(clssCount)
            dataXpred.append(Predictlabel)                    
        return dataXpred

In [ ]:
classes = np.unique(val_y)
t0 = time()
# Training the PCA + SOM
som = SelfOrganizingMaps(x = 30, y = 30, input_len = 784, sigma = 1.0, 
                         lr = 0.5, num_iteration= 5000, random_seed=56)
som.model.random_weights_init(train_x)
som.train(train_x)
print('training time: ', time() - t0)


training time:  107.10339426994324


In [ ]:
wt = som.model.winner(val_x)
print(wt.shape)

In [ ]:
t0 = time()
pred = som.predict(val_x, train_x, train_y, classes)
print('predict time: ', time() - t0)

predict time:  242.69993829727173


In [ ]:
print("NMI:", normalized_mutual_info_score(val_y, pred))
pred = np.array(pred)
print("ACC:", met.acc(val_y, pred))

NMI: 0.7012943835484433
ACC: 0.8254


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [11]:
class Encoder(nn.Module):
    def __init__(self, dims):
        super(Encoder, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(dims[0],dims[1]),        
            nn.ReLU(True),
            nn.Linear(dims[1],dims[2]),
            nn.ReLU(True),
            nn.Linear(dims[2],dims[3]),
            nn.ReLU(True),
            nn.Linear(dims[3],dims[4]),
            nn.Sigmoid(),
        )
 
    def forward(self,x):
        x=self.model(x)    
        return x
 
 
class Decoder(nn.Module):
    def __init__(self, dims):
        super(Decoder, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(dims[4],dims[3]),        
            nn.ReLU(True),
            nn.Linear(dims[3],dims[2]),
            nn.ReLU(True),
            nn.Linear(dims[2],dims[1]),
            nn.ReLU(True),
            nn.Linear(dims[1],dims[0]),
            nn.ReLU(True),
        )
    
    def forward(self,x):
        x=self.model(x)    
        return x
 
class Autoencoder(nn.Module):   
    def __init__(self, dims, init):        
        super(Autoencoder,self).__init__()
        self.encoder= Encoder(dims)
        self.decoder= Decoder(dims)
        if init is not None:
          self.initialize_weights(init)
 
    def forward(self, x):
        x=self.encoder(x)
        x=self.decoder(x)
        return x
    
    def initialize_weights(self, init):
        if init == "glorot_uniform":
          range = [0,2,4,6]
          for i in range:
            nn.init.xavier_normal_(self.encoder.model[i].weight.data)
            nn.init.xavier_normal_(self.decoder.model[i].weight.data)

In [ ]:
from time import time
epochs = 500
batch_size = 2048
if torch.cuda.is_available()==True:
    device="cuda"
else:
    device ="cpu"

dims=[train_x.shape[-1], 500, 500, 2000, 10]
AE = Autoencoder(dims, init=None)
AE.to(device)
criterion=nn.MSELoss(reduction='mean')
optimizer=optim.Adam(AE.parameters())
train = torch.Tensor(train_x)
trainloader=DataLoader(train_x, batch_size=batch_size)     

'''kwargs = {'num_workers': 1, 'pin_memory': True} if device=="cuda:0" else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)'''

t0 = time()
for epoch in range(epochs):
    train_loss = 0
    for i, (data)  in enumerate(trainloader):
        data = data.to(device)
        #Forward Pass
        output= AE(data)
        loss=criterion(output, data)
        #Backward Pass
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print("======> epoch: {}/{}, Loss:{}".format(epoch,epochs,train_loss))
print('Pretraining time: ', time() - t0)


In [ ]:
def eval_predict(model,loader):
  model.eval()
  all_predict = torch.tensor([], device=device)
  with torch.no_grad():
      for i, (data) in enumerate(loader):
          data = data.to(device)
          predict = model(data)
          all_predict = torch.cat((all_predict, predict), 0)
  return all_predict 
  
trainX = eval_predict(AE.encoder, trainloader).cpu().numpy()

In [ ]:
# Training the SOM without PCA initialization
t0 = time()
som = SelfOrganizingMaps(x = 30, y = 30, input_len = 10, sigma = 1.0, 
                         lr = 0.5, num_iteration= 5000, random_seed=56)
som.model.random_weights_init(trainX)
som.train(trainX)
print('training time: ', time() - t0)

training time:  55.02388286590576


In [ ]:
classes = np.unique(train_y)
t0 = time()
pred = som.predict(trainX, trainX, train_y, classes)
print('predict time: ', time() - t0)

predict time:  221.85195016860962


In [ ]:
print("NMI:", normalized_mutual_info_score(train_y, pred))
pred = np.array(pred)
print("ACC:", met.acc(train_y, pred))

NMI: 0.8000848249845998
ACC: 0.9038833333333334


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [12]:
class ClusteringLayer(nn.Module):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.
 
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """
    def __init__(self, n_clusters, n_features, weights=None, alpha=1.0):        
        super(ClusteringLayer, self).__init__()
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.n_features = n_features
        
        if weights is not None:
            self.weights = nn.Parameter(weights)
        else:
            weights = torch.Tensor(self.n_clusters, self.n_features)
            self.weights = nn.Parameter(weights)     
            
        nn.init.xavier_normal_(self.weights)
        
    def set_weights(self, weights):
        self.weights = nn.Parameter(weights)

    def forward(self, inputs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        input = inputs.clone().unsqueeze_(-1)
        input = input.reshape(input.size()[0],1,self.n_features)
        square = torch.square(input - self.weights)
        q = 1.0 / (1.0 + (square.sum(dim=2) / self.alpha))
        q = torch.pow(q, (self.alpha + 1.0) / 2.0)        
        q = torch.transpose(torch.transpose(q, 0, 1) / q.sum(dim=1), 0, 1)
        return q
    

In [13]:
class DEC(nn.Module):
    def __init__(self,
                 dims,
                 n_clusters=900,
                 alpha=1.0,
                 init='glorot_uniform'):
 
        super(DEC, self).__init__()        
        self.dims = dims
        self.n_features = dims[4]
        self.n_clusters = n_clusters
        self.alpha = alpha
        encoder = Encoder(self.dims)
        clustering_layer = ClusteringLayer(self.n_clusters, self.n_features)
        self.model = nn.Sequential()
        self.model.add_module("encoder", encoder)
        self.model.add_module("clustering_layer", clustering_layer)
 
    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
from torch.utils.data import TensorDataset
 
class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)
 
    # custom memory pinning method on custom type
    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self
 
def collate_wrapper(batch):
    return SimpleCustomBatch(batch)

In [40]:
class DEC_Module():
    def __init__(self,
                 dims,
                 n_clusters,
                 alpha=1.0,
                 init='glorot_uniform'):
        
      if torch.cuda.is_available()==True:
          self.device="cuda"
      else:
          self.device ="cpu"
      self.AE = Autoencoder(dims, init=init)
      self.dec = DEC(dims, n_clusters, init)
    
    def pretrain(self, x, y=None, optimizer='adam', epochs=200, batch_size=256, save_dir='results/temp'):
      print('...Pretraining...') 
      device = self.device
      self.AE.to(device)
      criterion=nn.MSELoss()
      optimizer=optim.Adam(self.AE.parameters())
      trainloader=DataLoader(x, batch_size=batch_size)     
      
      t0 = time()
      for epoch in range(epochs):
          train_loss = 0
          for i, (data)  in enumerate(trainloader):
              data = data.to(device)
              #Forward Pass
              output=self.AE(data)
              loss=criterion(output, data)
              #Backward Pass
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
              
          print("======> epoch: {}/{}, Loss:{}".format(epoch,epochs,loss.item()))
          if y is not None:
              self.print_on_epoch_end(y, epoch,epochs, trainloader)
      print('Pretraining time: ', time() - t0)
      self.set_encoder_weights(self.AE.encoder.state_dict())
      torch.save(self.AE.encoder.state_dict(), save_dir + '/encoder_weights.h5')
      print('Pretrained weights are saved to %s/encoder_weights.h5' % save_dir)
      self.pretrained = True
 
    def print_on_epoch_end(self,y, epoch, epochs, loader):            
      if epoch % int(epochs/10) != 0:
          return
      features = self.eval_predict(self.AE.encoder, loader).cpu().numpy()
 
      '''classes = np.unique(y)                    
      som = SelfOrganizingMaps(x = 30, y = 30, input_len = len(classes), sigma = 1.0, 
            lr = 0.5, num_iteration= 1,random_seed=56)
      som.model.random_weights_init(features)
      som.train(features)
      y_pred = som.predict(features, features, y, classes)'''
      km = KMeans(n_clusters=len(np.unique(y)), n_init=20, n_jobs=4)
      y_pred = km.fit_predict(features)
      y_pred = np.array(y_pred)
      print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
            % (met.acc(y, y_pred), met.nmi(y, y_pred)))       
 
    def extract_features(self, x):
      loader=DataLoader(x, batch_size=self.batch_size)                
      features = self.eval_predict(self.dec.AE.encoder, loader)
      features = features.cpu().numpy()
      return features
 
    def predict(self, loader): #Output from clustering layer 
      q = self.eval_predict(self.dec.model, loader)
      q = q.cpu().numpy()
      return q.argmax(1)
 
    @staticmethod
    def target_distribution(q):
      weight = torch.square(q) / q.sum(0)
      return torch.transpose(torch.transpose(weight,0,1)/weight.sum(1),0,1)
 
    def eval_predict(self, model,loader):
      device = self.device
      model.eval()
      all_predict = torch.tensor([], device=device)
      with torch.no_grad():
          for i, data in enumerate(loader):
              data = data.to(device)
              predict = model(data)
              all_predict = torch.cat((all_predict, predict), 0)
      return all_predict 

    def set_encoder_weights(self, state_dict):
      self.dec.model.encoder.load_state_dict(state_dict)
    
    def set_clustering_weights(self, weights):
      weights = torch.Tensor(weights)
      self.dec.model.clustering_layer.set_weights(weights.to(self.device))
    
    def train(self, x, y=None, maxiter=5e4, batch_size=256, tol=1e-3,
            update_interval=140, save_dir='./results/temp'):
      print('Update interval', update_interval)
      save_interval = x.shape[0] / batch_size * 5
      print('Save interval', save_interval)
      x = torch.Tensor(x)
      # Step 1: initialize cluster centers using SOM
      t1 = time()
      print('Initializing cluster centers with SOM.')
      self.dec.model.to(self.device)
      trainloader=DataLoader(x, batch_size=batch_size)
      #train_x = self.eval_predict(self.AE.encoder, trainloader).cpu().numpy()
      #self.set_encoder_weights(self.AE.encoder.state_dict())
      train_x = self.eval_predict(self.dec.model.encoder, trainloader).cpu().numpy()
      classes = np.unique(y)                    
      '''som = SelfOrganizingMaps(x = 30, y = 30, input_len = 10, sigma = 1.0, 
            lr = 0.5, num_iteration= 5000,random_seed=56)
      som.model.random_weights_init(train_x)
      som.train(train_x)
      som_weights = som.model.get_weights()
      som_weights = som_weights.reshape(900,10)
      self.set_clustering_weights(som_weights)'''

      kmeans = KMeans(n_clusters=len(classes), n_init=20)
      y_pred = kmeans.fit_predict(train_x)
      clster = kmeans.cluster_centers_
      self.set_clustering_weights(clster)
      
      #initial soft probability predictions
      q = self.eval_predict(self.dec.model, trainloader)
      #initial target distribution p
      p = self.target_distribution(q)
      p = p.cpu().numpy()
 
      #get class predictions
      #y_pred = som.predict(train_x,train_x,train_y, classes) 
      y_pred = np.array(y_pred)     
      y_pred_last = np.copy(y_pred)

      print("Initial accuracy km: %.4f" % met.acc(y, y_pred))
      print("Initial accuracy dec: %.4f" % met.acc(y, q.cpu().numpy().argmax(1)))

      # Step 2: deep clustering
      # logging file
      import csv
      logfile = open(save_dir + '/dec_log.csv', 'w')
      logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'loss'])
      logwriter.writeheader()
 
      loss = 0
      index = 0
      index_array = np.arange(x.shape[0])
      criterion=nn.KLDivLoss(reduction="batchmean")
      optimizer=optim.SGD(self.dec.model.parameters(), lr=0.01, momentum=0.9)
      
      '''print("DEC state_dict:")
      for param_tensor in self.dec.model.state_dict():
          print(param_tensor, "\t", self.dec.model.state_dict()[param_tensor].size())'''
 
      '''dataset = TensorDataset(x, p)
      loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
      loader_iterator = iter(loader)
      dataset = TensorDataset(x, p)
      loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
      loader_iterator = iter(loader)'''
      index = 0
      index_array = np.arange(x.shape[0])
      train_loss = 0
      for ite in range(int(maxiter)):
          if ite % update_interval == 0:
              q = self.eval_predict(self.dec.model, trainloader) #soft probability predictions on full dataset
              p = self.target_distribution(q)  # update the auxiliary target distribution p
              p = p.cpu().numpy()
              # evaluate the clustering performance
              y_pred = q.cpu().numpy()
              y_pred = y_pred.argmax(1)
              if y is not None:
                  acc = np.round(met.acc(y, y_pred), 5)
                  nmi = np.round(met.nmi(y, y_pred), 5)
                  ari = np.round(met.ari(y, y_pred), 5)
                  loss = np.round(train_loss, 5)
                  logdict = dict(iter=ite, acc=acc, nmi=nmi, ari=ari, loss=loss)
                  logwriter.writerow(logdict)
                  print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)
 
              # check stop criterion
              delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
              y_pred_last = np.copy(y_pred)
              if ite > 0 and delta_label < tol:
                  print('delta_label ', delta_label, '< tol ', tol)
                  print('Reached tolerance threshold. Stopping training.')
                  logfile.close()
                  break
          
          #Train on one batch
          idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
          input, target = torch.Tensor(x[idx]), torch.Tensor(p[idx])
          data = input.to(self.device)
          #Forward Pass
          output=self.dec.model(data)
          target = target.to(self.device)
          loss=criterion(output.log(), target)
          #Backward Pass
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          train_loss = loss.item()
          index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0 
                  
          # save intermediate model
          if ite % save_interval == 0:
              print('saving model to:', save_dir + '/KoDEC_model_' + str(ite) + '.h5')
              torch.save(self.dec.model.state_dict(), save_dir + '/KoDEC_model_' + str(ite) + '.h5')
 
          ite += 1
          
      # save the trained model
      logfile.close()
      print('saving model to:', save_dir + '/KoDEC_model_final.h5')
      torch.save(self.dec.model.state_dict(), save_dir + '/KoDEC_model_final.h5')
      return y_pred

In [43]:
#setting the hyper parameters
init = 'glorot_uniform'
pretrain_optimizer = 'adam'
dataset = 'mnist'
batch_size = 2048
maxiter = 2e4
tol = 0.001
save_dir = 'drive/DEC/pytorch/resultsnew'
 
import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
 
update_interval = 200
pretrain_epochs = 300

In [44]:
dec = DEC_Module(dims=[train_x.shape[-1], 500, 500, 2000, 10], n_clusters=10, init=init)

In [45]:
dec.pretrain(x=train_x, y=train_y, optimizer=pretrain_optimizer,
             epochs=pretrain_epochs, batch_size=batch_size,
             save_dir=save_dir)

...Pretraining...
======> epoch: 0/300, Loss:0.06911388784646988
        |==>  acc: 0.3534,  nmi: 0.2713  <==|
======> epoch: 1/300, Loss:0.05832405388355255
======> epoch: 2/300, Loss:0.053069014102220535
======> epoch: 3/300, Loss:0.043323591351509094
======> epoch: 4/300, Loss:0.03700592368841171
======> epoch: 5/300, Loss:0.032928407192230225
======> epoch: 6/300, Loss:0.030756615102291107
======> epoch: 7/300, Loss:0.029836876317858696
======> epoch: 8/300, Loss:0.028716588392853737
======> epoch: 9/300, Loss:0.02777022123336792
======> epoch: 10/300, Loss:0.029802318662405014
======> epoch: 11/300, Loss:0.026429975405335426
======> epoch: 12/300, Loss:0.02551441080868244
======> epoch: 13/300, Loss:0.025005271658301353
======> epoch: 14/300, Loss:0.024435052648186684
======> epoch: 15/300, Loss:0.023790856823325157
======> epoch: 16/300, Loss:0.0233159177005291
======> epoch: 17/300, Loss:0.023031534627079964
======> epoch: 18/300, Loss:0.022407079115509987
======> epoch: 19/300,

In [46]:
dec.dec.model.encoder.load_state_dict(torch.load("drive/DEC/pytorch/resultsnew/encoder_weights.h5"))

<All keys matched successfully>

In [ ]:
y_pred = dec.train(train_x, y=train_y, tol=tol, maxiter=maxiter, batch_size=batch_size,
                 update_interval=update_interval, save_dir=save_dir)